In [ ]:
# https://github.com/huggingface/smolagents

!pip install smolagents

In [ ]:
import streamlit as st
import pandas as pd
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, tool, LiteLLMModel, ToolCollection, Tool, ToolCallingAgent, ManagedAgent

# TODO: Replace hf_ZkEXVwIXXXXXXXXXXXXXXX with your Hugging Face token
model = HfApiModel(token="hf_ZkEXVwIXXXXXXXXXXXXXXX")

web_agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool()],
    model=model,
    max_steps=10,
)

managed_web_agent = ManagedAgent(
    agent=web_agent,
    name="search",
    description="Runs web searches for you. Give it your query as an argument.",
)

manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[managed_web_agent],
    additional_authorized_imports=['requests','json','pandas','matplotlib','xml.etree.ElementTree'],
)

In [ ]:
answer = 'N/A'
try:
    answer = manager_agent.run("Top 5 announcements at Snowflake Summit 2024 in JSON format. Only return the JSON formatted output as the response and nothing else.")
    st.json(answer)
except:
    st.write(answer)

In [ ]:
answer = manager_agent.run("Top 5 Cortex blogs from Snowflake Medium publication. Only return the reaponse  in a dataframe format as the response and nothing else.")
try:
    st.dataframe(pd.DataFrame(answer))
except:
    st.write(answer)

In [ ]:
class HFModelSnowflakeCortex(Tool):
    name="snowflake_cortex"
    description="""
    This tool summarizes text using Snowflake Cortex AI.
    """
    inputs = {
        "txt": {
            "type": "string",
            "description": "Text to analyze"
        }
    }
    output_type = "any"

    def forward(self,txt:str):
        import snowflake
        from snowflake.cortex import Complete
        response = {}
        prompt = f"""
                Summarize the text enclosed in ### in less than 200 words in JSON format and list out up to 3 highlights in JSON format ### {txt} ###. 
                Return only the JSON formatted output and nothing else.
        """
        response['summary'] = snowflake.cortex.Complete('claude-3-5-sonnet',prompt)
        return response

cortex_tool = HFModelSnowflakeCortex()

agent = CodeAgent(
    tools=[cortex_tool],
    model=model
)

answer= 'N/A'
try:
    answer = agent.run(
    """
        Snowflake ML is an integrated set of capabilities for end-to-end machine learning in a single platform on top of your governed data.
        
        For out-of-the-box ML workflows in SQL, the ready-to-use ML Functions can help shorten development time and democratize ML across your organization. 
        These functions let you train models for business use cases such as forecasting and anomaly detection without writing any code.
        
        For custom ML workflows in Python, data scientists and ML engineers can easily and securely develop and productionize scalable features and models without 
        any data movement, silos, or governance tradeoffs. The snowflake-ml-python library provides APIs for developing and deploying your Snowflake ML pipelines.
        
        To build and operationalize models, data scientists and ML engineers can leverage a suite of Snowflake ML features. 
        For model development, Snowflake ML Modeling APIs offer scalable data loading, feature engineering, and model training with distributed processing using 
        CPUs or GPUs. For ML Operations (ML Ops), Snowflake ML includes the Feature Store and Model Registry for centralized management of features and models in production.
        
        You can use Python APIs from the Snowpark ML library in Snowflake Notebooks, Snowsight worksheets. or your local Python IDE of choice.
    """)
    st.json(answer)
except:
    st.write(answer)